In [1]:
hkong = spark.table("hkongpercent_1_ord_currency_35acb_csv")
display(hkong)

In [2]:
from pyspark.ml.feature import VectorAssembler
X = hkong.columns[1:311]
assembler = VectorAssembler(inputCols=X, outputCol="features")

In [3]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression()
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")
# Import the tuning submodule
import pyspark.ml.tuning as tune
import numpy as np

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()

cv = tune.CrossValidator(estimator=lr,
                         estimatorParamMaps=grid,
                         evaluator=evaluator
                         )

In [4]:
from pyspark.sql.functions import col
scores=[]
target = hkong.columns[311:]
for i in target:
  #print i
  data = hkong.select(hkong.columns[:311] + [i])
  new_data = data.select(*[col(s).alias('label') if s ==i  else s for s in data.columns])
  new_data  = assembler.transform(new_data)  
  (training, test) = new_data.randomSplit([0.7, 0.3], seed = 100)  
  models = cv.fit(training)  
  best_lr = models.bestModel
  print i
  print(best_lr)    
  test_results = best_lr.transform(test)
  print (evaluator.evaluate(test_results))
  scores.append(evaluator.evaluate(test_results))

print(scores)
#print training.count()
#print test.count()
#display(new_data)


In [5]:
#not needed
from pyspark.sql.functions import col
new_data = data.select(*[col(s).alias('label') if s =='Target_0'  else s for s in data.columns])
display(new_data)

In [6]:
#not needed
new_data  =assembler.transform(new_data)
### Randomly split data into training and test sets. set seed for reproducibility
(training, test) = new_data.randomSplit([0.7, 0.3], seed = 100)
print training.count()
print test.count()

In [7]:
##Not needed
# Fit cross validation models
models = cv.fit(training)

# Extract the best model
best_lr = models.bestModel

# Print best_lr
print(best_lr)

In [8]:
#not needed
# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))